In [1]:
#
import sys

# from devolucao import get_df_devol
sys.path.append("..")
import DB
import workdays
import datetime
import pandas as pd
import numpy as np
import carteira_ibov
import taxas
import os
pd.options.mode.chained_assignment = None  # default='warn'
import config
import psycopg2
import pandas as pd
import workdays
import pyodbc
#
holidays_br = workdays.load_holidays("BR")
holidays_b3 = workdays.load_holidays("B3")

dt = datetime.date.today()
vcto_0 = dt
dt_pos = workdays.workday(dt, -1, holidays_br)


dt_1 = workdays.workday(dt, -1, holidays_b3)
dt_2 = workdays.workday(dt, -2, holidays_b3)
dt_3 = workdays.workday(dt, -3, holidays_b3)
dt_4 = workdays.workday(dt, -4, holidays_b3)

dt_next_1 = workdays.workday(dt, 1, holidays_b3)
vcto_1 = dt_next_1
dt_next_2 = workdays.workday(dt, 2, holidays_b3)
vcto_2 = dt_next_2
dt_next_3 = workdays.workday(dt, 3, holidays_b3)
vcto_3 = dt_next_3
dt_next_4 = workdays.workday(dt, 4, holidays_b3)
vcto_4 = dt_next_4
dt_next_5 = workdays.workday(dt, 5, holidays_b3)
vcto_5 = dt_next_5

dt_liq = workdays.workday(dt_1, 4, holidays_b3)


In [2]:
dt_1

datetime.date(2023, 5, 3)

In [3]:
db_conn_test = psycopg2.connect(
host=config.DB_TESTE_HOST,
dbname=config.DB_TESTE_NAME,
user=config.DB_TESTE_USER,
password=config.DB_TESTE_PASS)
db_conn_risk = psycopg2.connect(
host=config.DB_RISK_HOST,
dbname=config.DB_RISK_NAME,
user=config.DB_RISK_USER,
password=config.DB_RISK_PASS,
)

query = f"select * from tbl_alugueisconsolidados where dte_data='{dt_1.strftime('%Y-%m-%d')}'"

# query2 = f"""select contrato,cotliq, qtde, liquidacao from st_alugcustcorr where "data"= '{dt_1.strftime('%Y-%m-%d')}' """

query2 =  f"SELECT registro,st_alugcustcorr.cliente, st_alugcustcorr.corretora, 'T',vencimento,100*taxa, cotliq, reversor, codigo, st_alugcustcorr.contrato, (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) as saldo \
        from st_alugcustcorr left join st_alug_devolucao on st_alugcustcorr.cliente=st_alug_devolucao.cliente and st_alugcustcorr.contrato=st_alug_devolucao.contrato and dataliq>'{dt_1}' and dataliq<='{dt_liq.strftime('%Y-%m-%d')}' \
        where data='{dt_1.strftime('%Y-%m-%d')}' and qtde>0\
        group by registro,st_alugcustcorr.cliente, st_alugcustcorr.corretora, vencimento,taxa,cotliq, reversor, codigo, st_alugcustcorr.contrato\
        HAVING (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) <> 0 \
        UNION SELECT registro,st_alugcustcorr.cliente, st_alugcustcorr.corretora, 'D',vencimento,100*taxa, cotliq, reversor, codigo, st_alugcustcorr.contrato, (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) as saldo \
        from st_alugcustcorr left join st_alug_devolucao on st_alugcustcorr.contrato=st_alug_devolucao.contrato and dataliq>'{dt_1.strftime('%Y-%m-%d')}' and dataliq<='{dt_liq.strftime('%Y-%m-%d')}'\
        where data='{dt_1.strftime('%Y-%m-%d')}' and qtde<0\
        group by registro,st_alugcustcorr.cliente, st_alugcustcorr.corretora, vencimento,taxa,cotliq, reversor, codigo, st_alugcustcorr.contrato\
        HAVING (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) <> 0 order by codigo,vencimento " 


df = pd.read_sql(query, db_conn_test)
price = pd.read_sql(query2, db_conn_risk)
db_conn_risk.close()
db_conn_test.close()


In [4]:
price

,registro,cliente,corretora,?column?,vencimento,?column?,cotliq,reversor,codigo,contrato,saldo
0,2023-04-20,KAPITALO KAPPA MASTER FIM,BTG PACTUAL CTVM S/A,D,2023-06-01,32.61,22.46,TD,AALR3,2023042000424541570001-2,-500.0
1,2023-04-20,KAPITALO KAPPA MASTER FIM,BTG PACTUAL CTVM S/A,D,2023-06-01,32.61,22.46,TD,AALR3,2023042000424539690001-2,-500.0
2,2023-04-20,KAPITALO KAPPA MASTER FIM,BTG PACTUAL CTVM S/A,D,2023-06-01,32.61,22.46,TD,AALR3,2023042000424541790001-2,-1400.0
3,2023-05-02,KAPITALO KAPPA MASTER FIM,UBS BRASIL CCTVM S/A,D,2023-06-12,32.61,22.77,TD,AALR3,2023050200454612730001-1,-700.0
4,2023-03-22,KAPITALO ZETA MASTER FIM,NECTON INVESTIMENTOS S.A. CVMC,T,2023-05-04,5.75,13.99,TD,ABEV3,2023032200415016710001-2,15000.0
...,...,...,...,...,...,...,...,...,...,...,...
17470,2023-05-03,KAPITALO ZETA MASTER FIM,XP INVESTIMENTOS CCTVM S/A,T,2023-06-13,7.86,7.82,TD,YDUQ3,2023050300372156570001-2,4475.0
17471,2023-05-03,KAPITALO KAPPA MASTER FIM,ORAMA DTVM S A,T,2023-06-13,7.86,7.82,TD,YDUQ3,2023050300455344610001-1,2574.0
17472,2023-05-03,KAPITALO ZETA MASTER FIM,XP INVESTIMENTOS CCTVM S/A,T,2023-06-13,7.86,7.82,TD,YDUQ3,2023050300372156550001-2,6030.0
17473,2023-05-03,KAPITALO ZETA MASTER FIM,NECTON INVESTIMENTOS S.A. CVMC,T,2023-06-13,7.86,7.82,TD,YDUQ3,2023050300361917990001-2,2000.0


In [5]:
price.rename(columns={'cotliq':'preco','contrato':'str_numcontrato','cliente':'str_fundo'},inplace=True)
filt = price.loc[price['saldo']!=0][['preco','str_numcontrato']]

In [6]:
filt

,preco,str_numcontrato
0,22.46,2023042000424541570001-2
1,22.46,2023042000424539690001-2
2,22.46,2023042000424541790001-2
3,22.77,2023050200454612730001-1
4,13.99,2023032200415016710001-2
...,...,...
17470,7.82,2023050300372156570001-2
17471,7.82,2023050300455344610001-1
17472,7.82,2023050300372156550001-2
17473,7.82,2023050300361917990001-2


In [24]:
df = df.merge(filt,on=['str_numcontrato'],how='inner')

In [25]:
df.fillna(0,inplace=True)
df = df[(df['str_mesa'].isin(['Kapitalo 11.1','Kapitalo 1.0'])) & (df['dbl_quantidade']!=0)]

In [26]:
df

,dte_data,str_fundo,str_mercado,str_codigo,str_serie,str_mesa,str_estrategia,str_numcontrato,dbl_quantidade,preco
88,2023-04-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 11.1,Bolsa 2,2023033100431486540001-2,5303.0,7.17
89,2023-04-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 11.1,Bolsa 2,2023033100431486530001-2,2243.0,7.17
90,2023-04-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 11.1,Bolsa 2,2023033100431486510001-2,4131.0,7.17
91,2023-04-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 1.0,Bolsa 2,2023033100431486510001-2,14600.0,7.17
92,2023-04-03,KAPITALO ZETA MASTER FIM,Emprestimo RV,YDUQ,"YDUQ3-T-7,89-20230515-BALCAO",Kapitalo 11.1,Bolsa 2,2023033100431149830001-2,800.0,7.17
...,...,...,...,...,...,...,...,...,...,...
32966,2023-04-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,VALE,"VALE3-T-0,22-20230504-BALCAO",Kapitalo 11.1,Arbitragem Aluguel,2023032200442039770001-1,120.0,82.34
32967,2023-04-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,VALE,"VALE3-T-0,22-20230504-BALCAO",Kapitalo 11.1,Arbitragem Aluguel,2023032200442039760001-1,57.0,82.34
32968,2023-04-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,VALE,"VALE3-T-0,22-20230504-BALCAO",Kapitalo 11.1,Arbitragem Aluguel,2023032200442039730001-1,100.0,82.34
32969,2023-04-03,KAPITALO K10 PREV MASTER FIM,Emprestimo RV,VALE,"VALE3-T-0,22-20230504-BALCAO",Kapitalo 11.1,Arbitragem Aluguel,2023032200442039710001-1,93.0,82.34


In [27]:
check = price[['str_fundo','str_numcontrato','saldo']].merge(df[['str_fundo','str_numcontrato','dbl_quantidade']].drop_duplicates().groupby(['str_fundo','str_numcontrato']).agg({'dbl_quantidade':sum}).reset_index().drop_duplicates(),on=['str_fundo','str_numcontrato'],how='inner')

In [28]:
check.loc[(check['dbl_quantidade']!=check['saldo']) & (check['str_fundo']=='KAPITALO KAPPA MASTER FIM')]


,str_fundo,str_numcontrato,saldo,dbl_quantidade
1073,KAPITALO KAPPA MASTER FIM,2023021600428949560001-1,500.0,250.0


In [29]:

df['codigo'] = df['str_serie'].apply(lambda x: x.split('-')[0])
df['taxa'] = df['str_serie'].apply(lambda x: x.split('-')[2].replace(',','.'))
df['tipo'] = df['str_serie'].apply(lambda x: x.split('-')[1])
df['modalidade'] = df['str_serie'].apply(lambda x: x.split('-')[4])
df['vencimento'] = df['str_serie'].apply(lambda x: x.split('-')[3])
df['volume'] = df['dbl_quantidade'].astype(float)*df['taxa'].astype(float)
df = df.drop_duplicates()
ctos_btc = df
vencimentos = df



In [30]:
df = df.groupby(['str_fundo','str_mesa','str_estrategia','codigo','tipo']).agg({'dbl_quantidade':sum,'volume':sum}).reset_index()

In [31]:
df['taxa media'] = (df['volume']/df['dbl_quantidade']).apply(lambda x: round(x,2))

In [32]:
btc = pd.pivot_table(df,index=['str_fundo','str_mesa','str_estrategia','codigo'],columns=['tipo'],values=['dbl_quantidade','taxa media']).reset_index().fillna(0)

In [33]:
btc.columns = ['fundo',	'mesa',	'str_estrategia','codigo','DOADO','TOMADO','TAXA DOADORA','TAXA TOMADORA']

In [34]:
btc[btc['codigo']=='AMER3']

,fundo,mesa,str_estrategia,codigo,DOADO,TOMADO,TAXA DOADORA,TAXA TOMADORA
9,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,AMER3,0.0,35594.0,0.0,13.56
75,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Bolsa 2,AMER3,0.0,93247.0,0.0,12.34
161,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,CashCarry,AMER3,0.0,88626.0,0.0,10.70
249,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,MM,AMER3,0.0,2900.0,0.0,12.34
297,KAPITALO KAPPA PREV MASTER FIM,Kapitalo 1.0,Bolsa 2,AMER3,0.0,2034.0,0.0,15.23
343,KAPITALO KAPPA PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,AMER3,0.0,21538.0,0.0,13.40
385,KAPITALO SIGMA LLC,Kapitalo 1.0,Bolsa 2,AMER3,0.0,201.0,0.0,8.22
428,KAPITALO SIGMA LLC,Kapitalo 11.1,Bolsa 2,AMER3,0.0,100.0,0.0,8.22
473,KAPITALO ZETA MASTER FIA,Kapitalo 1.0,Bolsa 2,AMER3,0.0,1417.0,0.0,8.45
523,KAPITALO ZETA MASTER FIA,Kapitalo 11.1,Bolsa 2,AMER3,0.0,708.0,0.0,8.45


In [35]:
import workdays
holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')
dt = datetime.date.today()


dt_1 = workdays.workday(dt, -1, holidays_b3)

dt_1 = workdays.workday(dt, -1, holidays_b3)
vcto_0 = dt.strftime('%d/%m/%Y')
dt_pos = workdays.workday(dt, -1, holidays_br)
venc_interna = workdays.workday(dt, 1, holidays_br)



df_pos = DB.get_equity_positions_mesas(None,dt_1)


df_pos = pd.DataFrame(df_pos[['str_fundo','str_mesa','str_estrategia',"regexp_replace", "sum"]])
df_pos.rename(columns={"regexp_replace": "codigo", "sum": "position","str_fundo":"fundo",'str_mesa':'mesa'}, inplace=True)


df_pos = df_pos[ (df_pos['mesa'].isin(['Kapitalo 11.1','Kapitalo 1.0'])) & ~(df_pos['fundo'].isin(['KAPITALO CLASS B', 'KAPITALO CLASS K', 'KAPITALO CLASS OMEGA']))]

In [36]:
mapa = df_pos.merge(btc,on=['fundo','mesa','str_estrategia','codigo'],how='outer').fillna(0)

# Incluindo Internas K11

In [37]:
import pandas as pd
k11 = pd.read_excel('internas.xlsx')

In [38]:
k11['codigo'] = k11['SERIE'].apply(lambda x: x.split('-')[0])
k11['taxa'] = k11['SERIE'].apply(lambda x: x.split('-')[2].replace(',','.'))
k11['tipo'] = k11['SERIE'].apply(lambda x: x.split('-')[1])
k11['modalidade'] = k11['SERIE'].apply(lambda x: x.split('-')[4])
k11['vencimento'] = k11['SERIE'].apply(lambda x: x.split('-')[3])
k11['volume'] = k11['NOTIONAL'].astype(float)*k11['taxa'].astype(float)

In [39]:


k11 = k11.rename(columns={'ALOCACAO':'str_fundo','MESA':'str_mesa','ESTRATEGIA':'str_estrategia','NOTIONAL':'dbl_quantidade'})

In [40]:
k11 = k11.groupby(['str_fundo','str_mesa','str_estrategia','codigo','tipo']).agg({'dbl_quantidade':sum,'volume':sum}).reset_index()

In [41]:
k11 = pd.pivot_table(k11,index=['str_fundo','str_mesa','str_estrategia','codigo'],columns='tipo',values='dbl_quantidade').reset_index().fillna(0)

In [42]:
k11.rename(columns={'D':'interna DOADO','T':'interna TOMADO'},inplace=True)

In [43]:
k11.columns = ['fundo',	'mesa',	'str_estrategia','codigo','interna DOADO','interna TOMADO']

# Incluindo Internas Mesas

In [44]:
mesa = pd.read_excel('mesa.xlsx')

In [45]:
mesa = pd.pivot_table(mesa,index=['str_fundo','str_mesa','str_estrategia','codigo'],columns='tipo',values='lote estrategia').reset_index()

In [46]:
mesa['mesas DOADO'] = mesa['D']*-1
mesa['mesas TOMADO'] = mesa['T']*-1

KeyError: 'D'

In [ ]:
mesa = mesa[['str_fundo','str_mesa','str_estrategia','codigo','mesas DOADO','mesas TOMADO']].fillna(0)

In [ ]:
mesa.columns = ['fundo','mesa',	'str_estrategia','codigo','mesas DOADO','mesas TOMADO']

# Internas sinteticas + mapa

In [47]:
mapa = mapa.merge(k11,on=['fundo',	'mesa',	'str_estrategia','codigo'],how='outer').merge(mesa,on=['fundo',	'mesa',	'str_estrategia','codigo'],how='outer').fillna(0)

KeyError: 'fundo'

In [31]:
mapa

,fundo,mesa,str_estrategia,codigo,position,DOADO,TOMADO,TAXA DOADORA,TAXA TOMADORA,interna DOADO,interna TOMADO,mesas DOADO,mesas TOMADO
0,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,AALR3,6942.0,-177.0,0.0,3.82,0.00,0.0,0.0,0.0,0.0
1,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,AERI3,-31929.0,0.0,36830.0,0.00,1.09,0.0,0.0,0.0,0.0
2,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,AESB3,11068.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
3,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,AGRO3,8280.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
4,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,ALUP11,3960.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,KAPITALO ZETA MASTER FIA,Kapitalo 11.1,Bolsa 2,USIM5,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,100.0
1057,KAPITALO ZETA MASTER FIM,Kapitalo 1.0,Bolsa 2,CPLE6,0.0,0.0,0.0,0.00,0.00,0.0,0.0,-34600.0,0.0
1058,KAPITALO ZETA MASTER FIM,Kapitalo 1.0,Bolsa 2,USIM5,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,12600.0
1059,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,Bolsa 2,CPLE6,0.0,0.0,0.0,0.00,0.00,0.0,0.0,-34600.0,0.0


# Vencimentos

In [48]:

vencimentos['vencimento'] = vencimentos['vencimento'].apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d').date())

In [49]:
aux  = vencimentos.loc[(vencimentos['vencimento']>= dt) & (vencimentos['vencimento']< dt_next_5)]

In [50]:
aux = aux.groupby(['str_fundo','codigo','str_mesa','str_estrategia','vencimento']).agg({'dbl_quantidade':sum}).reset_index()

In [51]:
venc = pd.pivot_table(aux,index=['str_fundo','codigo','str_mesa','str_estrategia'],columns='vencimento',values='dbl_quantidade').reset_index().fillna(0)

In [52]:
venc.rename(columns={'str_fundo':'fundo','str_mesa':'mesa'},inplace=True)

In [53]:
mapa = mapa.merge(venc,on=['fundo','codigo','mesa','str_estrategia'],how='outer').fillna(0)

In [56]:
venc[venc['fundo']=='KAPITALO ZETA MASTER FIM']

vencimento,fundo,codigo,mesa,str_estrategia,2023-04-04,2023-04-05,2023-04-06,2023-04-10,2023-04-11
155,KAPITALO ZETA MASTER FIM,AERI3,Kapitalo 1.0,Bolsa 2,0.0,0.0,13397.0,0.0,0.0
156,KAPITALO ZETA MASTER FIM,AMAR3,Kapitalo 1.0,Bolsa 2,45158.0,0.0,0.0,0.0,100000.0
157,KAPITALO ZETA MASTER FIM,AMAR3,Kapitalo 11.1,Bolsa 2,8031.0,0.0,0.0,0.0,0.0
158,KAPITALO ZETA MASTER FIM,ANIM3,Kapitalo 1.0,Bolsa 2,0.0,0.0,0.0,0.0,35013.0
159,KAPITALO ZETA MASTER FIM,ANIM3,Kapitalo 11.1,Bolsa 2,0.0,0.0,0.0,0.0,2431.0
160,KAPITALO ZETA MASTER FIM,BPAN4,Kapitalo 1.0,Bolsa 2,6900.0,0.0,0.0,0.0,6300.0
161,KAPITALO ZETA MASTER FIM,BPAN4,Kapitalo 11.1,Bolsa 2,714.0,0.0,0.0,0.0,4850.0
162,KAPITALO ZETA MASTER FIM,BRKM5,Kapitalo 1.0,Bolsa 2,4080.0,0.0,1587.0,0.0,0.0
163,KAPITALO ZETA MASTER FIM,BRKM5,Kapitalo 11.1,Bolsa 2,390.0,0.0,313.0,0.0,0.0
164,KAPITALO ZETA MASTER FIM,CASH3,Kapitalo 1.0,Bolsa 2,183569.0,0.0,0.0,0.0,0.0


# Movimentações

In [65]:
def get_equity_trades():
    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)    
    query=f"SELECT * FROM tbl_auxboletas1 where dte_data > '{dt_3.strftime('%Y-%m-%d')}' and ((str_mesa='Kapitalo 11.1' and str_mercado='Acao') or (str_mesa='Kapitalo 1.0' and  str_estrategia = 'Bolsa 2' and str_mercado='Acao') ) and str_fundo not in ('KAPITALO CLASS B', 'KAPITALO CLASS OMEGA') and dte_data != '{dt.strftime('%Y-%m-%d')}'" 
    try:
        df =pd.read_sql(query,db_conn_test)
    except:
        pass
    db_conn_test.close()

    return df

In [66]:
trades = get_equity_trades()

In [67]:
trades  = trades.groupby(['str_fundo','str_serie','str_mesa','str_estrategia','dte_data']).agg({'dbl_lote':sum}).reset_index()

In [68]:
trades = pd.pivot_table(trades,index=['str_fundo','str_serie','str_mesa','str_estrategia'],columns='dte_data',values='dbl_lote').reset_index()


In [69]:
trades['str_serie'] = trades['str_serie'].apply(lambda x: x.replace(' BZ EQUITY',''))
trades.rename(columns={
    dt_2:'mov_0',
    dt_1:'mov_1',
    'str_fundo':'fundo',
    'str_mesa':'mesa',
    'str_serie':'codigo'
    },inplace=True
)

In [62]:
mapa = mapa.merge(trades,on=['fundo','codigo','mesa','str_estrategia'],how='outer').fillna(0)

In [70]:
trades

dte_data,fundo,codigo,mesa,str_estrategia,mov_0,mov_1
0,KAPITALO K10 PREV MASTER FIM,ELET3,Kapitalo 11.1,Arbitragem Aluguel,-1446.0,8620.0
1,KAPITALO K10 PREV MASTER FIM,GGBR4,Kapitalo 11.1,Arbitragem Aluguel,NaN,63.0
2,KAPITALO K10 PREV MASTER FIM,NUBR33,Kapitalo 11.1,Arbitragem Aluguel,NaN,-2649.0
3,KAPITALO K10 PREV MASTER FIM,SUZB3,Kapitalo 11.1,Arbitragem Aluguel,-4915.0,-846.0
4,KAPITALO K10 PREV MASTER FIM,VALE3,Kapitalo 11.1,Arbitragem Aluguel,-3053.0,47.0
...,...,...,...,...,...,...
119,KAPITALO KAPPA PREV MASTER FIM,ELET3,Kapitalo 11.1,Arbitragem Aluguel,-685.0,4045.0
120,KAPITALO KAPPA PREV MASTER FIM,GGBR4,Kapitalo 11.1,Arbitragem Aluguel,NaN,30.0
121,KAPITALO KAPPA PREV MASTER FIM,NUBR33,Kapitalo 11.1,Arbitragem Aluguel,NaN,-1248.0
122,KAPITALO KAPPA PREV MASTER FIM,SUZB3,Kapitalo 11.1,Arbitragem Aluguel,-2329.0,-338.0


In [45]:
query_ibotz = f" select str_fundo,str_mesa,str_estrategia,str_codigo,str_serie, sum(dbl_quantidade) as dbl_quantidade from ibotz.tbl_boletasalugueis_ibotz where dte_data='{dt.strftime('%Y-%m-%d')}' and str_mesa in ('Kapitalo 11.1','Kapitalo 1.0') and str_mercado='Emprestimo RV' group by str_fundo,str_mesa,str_estrategia,str_codigo,str_serie"
db_conn_test = psycopg2.connect(
host=config.DB_TESTE_HOST,
dbname=config.DB_TESTE_NAME,
user=config.DB_TESTE_USER,
password=config.DB_TESTE_PASS)

btc_ibotz = pd.read_sql(query_ibotz,db_conn_test)
db_conn_test.close()

In [46]:
btc_ibotz['codigo'] = btc_ibotz['str_serie'].apply(lambda x: x.split('-')[0])
btc_ibotz['tipo'] = btc_ibotz['str_serie'].apply(lambda x: x.split('-')[1])
btc_ibotz = btc_ibotz.groupby(['str_fundo','str_mesa','str_estrategia','codigo','tipo']).agg({'dbl_quantidade':sum}).reset_index()


In [47]:
btc_ibotz

,str_fundo,str_mesa,str_estrategia,codigo,tipo,dbl_quantidade
0,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,BRFS3,T,5856.0
1,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,CVCB3,T,238.0
2,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Bolsa 2,BBDC4,T,3600.0
3,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Bolsa 2,RAIZ4,T,4629.0
4,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Bolsa 2,SEQL3,T,1787.0
...,...,...,...,...,...,...
123,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,Bolsa 2,MGLU3,T,12444.0
124,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,Bolsa 2,MRFG3,T,18308.0
125,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,Bolsa 2,MRVE3,T,18696.0
126,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,Bolsa 2,SBFG3,T,14489.0


In [48]:
btc_ibotz = pd.pivot_table(btc_ibotz,columns='tipo',index=['str_fundo','str_mesa','codigo','str_estrategia'],values='dbl_quantidade').reset_index().fillna(0).rename(
    columns=
    {'str_fundo':'fundo',
    'str_mesa':'mesa',
    'D':'trade_doado',
    'T':'trade_tomado'}
    )

In [49]:
btc_ibotz

tipo,fundo,mesa,codigo,str_estrategia,trade_tomado
0,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,BRFS3,Bolsa 2,5856.0
1,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,CVCB3,Bolsa 2,238.0
2,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,BBDC4,Bolsa 2,3600.0
3,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,RAIZ4,Bolsa 2,4629.0
4,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,SEQL3,Bolsa 2,1787.0
...,...,...,...,...,...
123,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,MGLU3,Bolsa 2,12444.0
124,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,MRFG3,Bolsa 2,18308.0
125,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,MRVE3,Bolsa 2,18696.0
126,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,SBFG3,Bolsa 2,14489.0


In [50]:
mapa = mapa.merge(btc_ibotz,on=['fundo','mesa', 'str_estrategia','codigo'],how='outer').fillna(0)

In [51]:
mapa["PendRecallD1"] = 0
mapa["PendRecallD2"] = 0
mapa["PendRecallD3"] = 0
mapa['mov_2'] = 0

In [52]:
if not 'trade_doado' in mapa.columns:
    mapa['trade_doado'] = 0 
if not 'trade_tomado' in mapa.columns:
    mapa['trade_tomado'] = 0

In [53]:

mapa["pos_doada"] = mapa["DOADO"] + mapa["trade_doado"] + mapa['interna DOADO'] + mapa['mesas DOADO']
mapa["pos_tomada"] = mapa["TOMADO"] + mapa["trade_tomado"] + mapa['interna TOMADO'] + mapa['mesas TOMADO']
mapa["net_alugado"] = mapa["pos_doada"] + mapa["pos_tomada"]
mapa["custodia_aux"] = mapa["position"] + mapa["net_alugado"] - mapa["mov_0"] - mapa["mov_1"]

# mapa['custodia_janela'] = Saldo doador - Mov_0 (dia atual)
try:
    mapa["custodia_0"] = mapa["custodia_aux"] - mapa[vcto_0] + mapa["mov_0"]
except:
    mapa["custodia_0"] = mapa["custodia_aux"] + mapa["mov_0"]
mapa["custodia_0"].fillna(0, inplace=True)

try:
    mapa["custodia_1"] = mapa["custodia_0"] + mapa["mov_1"] - mapa[vcto_1] + mapa["PendRecallD1"]
except:
    mapa["custodia_1"] = mapa["custodia_0"] + mapa["mov_1"] + mapa["PendRecallD1"]
    
try:
    mapa["custodia_2"] = mapa["custodia_1"] - mapa[vcto_2] + mapa["PendRecallD2"] + mapa["mov_2"]
except:
    mapa["custodia_2"] = mapa["custodia_1"] + mapa["PendRecallD2"] + mapa["mov_2"]
    
mapa["custodia_3"] = mapa["custodia_2"] - mapa[vcto_3] + mapa["PendRecallD3"]

mapa["to_borrow_0"] = np.minimum(0, mapa["custodia_0"])
mapa["to_borrow_0"].fillna(0, inplace=True)
mapa["to_borrow_1"] = np.minimum(0, mapa["custodia_1"] - mapa["to_borrow_0"])
mapa["to_borrow_1"].fillna(0, inplace=True)
mapa["to_borrow_2"] = np.minimum(
    0, mapa["custodia_2"] - mapa["to_borrow_0"] - mapa["to_borrow_1"]
)
mapa["to_borrow_2"].fillna(0, inplace=True)
mapa["to_borrow_3"] = np.minimum(
    0, mapa["custodia_3"] - mapa["to_borrow_0"] - mapa["to_borrow_1"] - mapa["to_borrow_2"]
)

mapa["to_borrow_3"].fillna(0, inplace=True)



In [54]:
mapa["custodia_exaluguel"] = (
    mapa["custodia_aux"]
    - mapa["net_alugado"]
    + np.minimum.reduce(
        [
            mapa["mov_0"],
            mapa["mov_0"] + mapa["mov_1"],
            mapa["mov_0"] + mapa["mov_1"] + mapa["mov_2"],
        ]
    )
)

mapa["devol_tomador"] = np.minimum(
    -np.minimum(mapa["custodia_exaluguel"], 0) - mapa["pos_tomada"], 0
)
mapa.loc[mapa['to_borrow_0']<0,'devol_tomador'] = 0

mapa["devol_doador"] = np.maximum(
    -np.maximum(mapa["custodia_exaluguel"], 0)
    - mapa["pos_doada"]
    - mapa["PendRecallD1"]
    - mapa["PendRecallD2"]
    - mapa["PendRecallD3"],
    0,
)
mapa["devol_tomador"].fillna(0, inplace=True)
mapa["devol_doador"].fillna(0, inplace=True)

In [55]:
ind_devol =mapa[mapa['devol_tomador']!=0][['fundo','mesa','str_estrategia','codigo','custodia_0','devol_tomador']]
ajuste = mapa[mapa['to_borrow_0']!=0][['fundo','mesa','str_estrategia','codigo','to_borrow_0']]
sinal_ajuste = ajuste[['fundo','mesa','codigo','to_borrow_0']]
ind_devol['devol_tomador'] = -ind_devol['devol_tomador']
ind_devol = ind_devol.merge(sinal_ajuste,on=['fundo','mesa','codigo'],how='inner').drop_duplicates()

# Ajuste de contratos

In [56]:
ind_devol['to_borrow_0'] = -ind_devol['to_borrow_0']
ind_devol = ind_devol.groupby(['fundo', 'mesa', 'str_estrategia', 'codigo', 'custodia_0','devol_tomador']).agg({'to_borrow_0':sum}).reset_index()
ind_devol['indicativo devol'] = ind_devol.apply(lambda row: min(row['to_borrow_0'],max(row['custodia_0'],row['devol_tomador'])),axis=1)

ind_devol = ind_devol.drop_duplicates()
ctos_btc.rename(columns={'str_fundo':'fundo','str_mesa':'mesa'},inplace=True)
devol = ctos_btc.merge(ind_devol, on=["fundo",'mesa','str_estrategia',"codigo"], how="inner")
devol = devol.drop_duplicates()
devol = devol.loc[(devol['tipo']=='T') & (devol['dbl_quantidade']>0)]

devol = devol.sort_values('codigo')
devol['Devolucao']=0
ind_devol = ind_devol.groupby(['fundo', 'mesa', 'str_estrategia', 'codigo']).agg({'indicativo devol':sum}).reset_index()


In [57]:
devol = devol[['dte_data', 'fundo', 'str_mercado', 'str_codigo', 'str_serie', 'mesa',
       'str_estrategia', 'str_numcontrato', 'dbl_quantidade', 'preco',
       'codigo','Devolucao']].drop_duplicates()

In [58]:
for i, row in devol.iterrows():
        saldo = ind_devol.loc[(ind_devol['fundo']==row['fundo'])&(ind_devol['codigo']==row['codigo'])&(ind_devol['mesa']==row['mesa'])&(ind_devol['str_estrategia']==row['str_estrategia']),'indicativo devol'].sum()
        devol.loc[(devol['fundo']==row['fundo'])&(devol['codigo']==row['codigo'])&(devol['mesa']==row['mesa'])& (devol['str_estrategia']==row['str_estrategia'] )& (devol['str_numcontrato']==row['str_numcontrato']),'Devolucao'] = min(row['dbl_quantidade'],saldo)
        ind_devol.loc[(ind_devol['fundo']==row['fundo'])&(ind_devol['codigo']==row['codigo'])&(ind_devol['mesa']==row['mesa'])&(ind_devol['str_estrategia']==row['str_estrategia']),'indicativo devol'] = round( saldo -min(row['dbl_quantidade'],saldo) ,2)

In [59]:
rebal_tomadores = devol[devol['Devolucao']!=0]

In [60]:
## Preciso realizar com contratos doadores
ind_devol[ind_devol['indicativo devol']!=0]

,fundo,mesa,str_estrategia,codigo,indicativo devol
7,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Bolsa 2,BBDC4,3600.0
11,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Bolsa 2,BRKM5,200.0
31,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Bolsa 2,GGBR4,2418.0
61,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Bolsa 2,VALE3,7204.0
68,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,CashCarry5,BOVA11,5837.0
79,KAPITALO ZETA MASTER FIA,Kapitalo 1.0,Bolsa 2,BOVA11,3836.0


In [61]:
alloc_tomadores = rebal_tomadores

In [62]:
alloc_tomadores['dbl_quantidade'] = alloc_tomadores['Devolucao'] 
alloc_tomadores = alloc_tomadores[alloc_tomadores['dbl_quantidade']!=0]

In [63]:
alloc_tomadores.rename(columns={'Devolucao':'Alocacao'},inplace= True)

In [64]:
alloc_tomadores['estrategia target'] = 0

In [65]:
map_ajuste = ajuste

In [66]:
map_ajuste['to_borrow_0'] = -map_ajuste['to_borrow_0']

In [19]:
DB.get_taxasalugueis(None)[['tckrsymb','takravrgrate']].to_clipboard()

In [68]:
aux = pd.DataFrame()
alloc_tomadores["dbl_quantidade"] = alloc_tomadores["dbl_quantidade"].apply(
		lambda x: [1 for i in range(abs(int(x)))]
		if x > 0
		else [-1 for i in range(abs(int(x)))]
	)
alloc_tomadores = alloc_tomadores.explode("dbl_quantidade")
for i, row in map_ajuste.iterrows():
    for x, x_row in alloc_tomadores.iterrows():
        if ((row['codigo']=='ABEV3')):
            if x_row['estrategia target']==0:
                saldo = map_ajuste.loc[(map_ajuste['fundo']==row['fundo'])&(map_ajuste['codigo']==row['codigo'])&(map_ajuste['mesa']==row['mesa'])&(map_ajuste['str_estrategia']==row['str_estrategia']),'to_borrow_0'].sum()
            
                alloc_tomadores.loc[(alloc_tomadores['fundo']==row['fundo'])&
                (alloc_tomadores['fundo']==x_row['fundo'])&
                (alloc_tomadores['codigo']==row['codigo'])&
                (alloc_tomadores['codigo']==x_row['codigo'])&
                (alloc_tomadores['mesa']==row['mesa'])&
                (alloc_tomadores['mesa']==x_row['mesa'])&
                (alloc_tomadores['str_estrategia']==x_row['str_estrategia'])&
                (alloc_tomadores['str_numcontrato']==x_row['str_numcontrato']),'volta'] = min(x_row['dbl_quantidade'],saldo)

                alloc_tomadores.loc[(alloc_tomadores['fundo']==row['fundo'])&
                (alloc_tomadores['fundo']==x_row['fundo'])&
                (alloc_tomadores['codigo']==row['codigo'])&
                (alloc_tomadores['codigo']==x_row['codigo'])&
                (alloc_tomadores['mesa']==row['mesa'])&
                (alloc_tomadores['mesa']==x_row['mesa'])&
                (alloc_tomadores['str_estrategia']==x_row['str_estrategia'])&
                (alloc_tomadores['str_numcontrato']==x_row['str_numcontrato']),'estrategia target'] = row['str_estrategia']
                
                
                map_ajuste.loc[(map_ajuste['fundo']==row['fundo'])&(map_ajuste['codigo']==row['codigo'])&(map_ajuste['mesa']==row['mesa'])&(map_ajuste['str_estrategia']==row['str_estrategia']),'to_borrow_0'] = round( saldo -min(x_row['dbl_quantidade'],saldo) ,2)

   






In [ ]:
aux = alloc_tomadores[alloc_tomadores['codigo']=='ABEV3']

In [ ]:
alloc_tomadores[alloc_tomadores['estrategia target']!=0].groupby(['str_codigo','str_estrategia']).agg({'volta':sum})

,,volta
str_codigo,str_estrategia,
ABEV,CashCarry,900.0


In [ ]:
aux[aux['volta']!=0].groupby('str_estrategia').agg({'volta':sum})

,volta
str_estrategia,
CashCarry,900.0


In [ ]:
aux["dbl_quantidade"] = aux["dbl_quantidade"].apply(
		lambda x: [1 for i in range(abs(int(x)))]
		if x > 0
		else [-1 for i in range(abs(int(x)))]
	)
aux = aux.explode("dbl_quantidade")

In [ ]:
ctos_btc[]